In [1]:
!pip install -U -q "google"
!pip install -U -q "google.genai"
import os

In [5]:
import sys
import os

# Add the parent 'data' folder to sys.path
parent_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
if parent_data_path not in sys.path:
    sys.path.append(parent_data_path)

print(f"Mounted data folder at: {parent_data_path}")
# Now you can import modules or access files from the 'data' folder


Mounted data folder at: /workspaces/LLM-AI-AGENT-Demo/data


In [9]:
!pip install -U -q "google"
!pip install -U -q "google.genai"
!pip install -U -q "chromadb"
!pip install -U -q "python-dotenv"
!pip install -U -q "langchain_community"
!pip install -U -q "langchain_text_splitters"
!pip install -U -q "pypdf"
!pip install -U -q "google.generativeai"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [7]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb

# setting the environment

DATA_PATH = parent_data_path
CHROMA_PATH = r"chroma_db"


chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)

collection = chroma_client.get_or_create_collection(name="RAG")

# loading the document

loader = PyPDFDirectoryLoader(DATA_PATH)

raw_documents = loader.load()


print(f"Loaded {len(raw_documents)} documents from {DATA_PATH}")

# splitting the document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_documents(raw_documents)

# preparing to be added in chromadb

documents = []
metadata = []
ids = []

i = 0

for chunk in chunks:
    documents.append(chunk.page_content)
    ids.append("ID"+str(i))
    metadata.append(chunk.metadata)

    i += 1

# adding to chromadb


collection.upsert(
    documents=documents,
    metadatas=metadata,
    ids=ids
)

Loaded 783 documents from /workspaces/LLM-AI-AGENT-Demo/data


/home/codespace/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:06<00:00, 12.1MiB/s]


In [12]:
import chromadb
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()

# setting the environment

DATA_PATH = "parent_data_path"
CHROMA_PATH = r"chroma_db"

chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)

collection = chroma_client.get_or_create_collection(name="RAG")

# Configure Gemini API
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('gemini-1.5-flash-latest')


user_query = input("What do you want to know about RAG Techniques?\n\n")

results = collection.query(
    query_texts=[user_query],
    n_results=20
)

#print(results['documents'])
#print(results['metadatas'])

system_prompt = """
You are a health and wellness coach. You answer questions about Health and wellness.
But you only answer based on knowledge I'm providing you. You don't use your internal
knowledge and you don't make thins up.
If you don't know the answer, just say: I don't know
--------------------
The data:
"""+str(results['documents'])+"""
"""

#print(system_prompt)

response = model.generate_content(
    contents = [
        {"role":"user","parts":[{"text":system_prompt}]},
        {"role":"model","parts":[{"text":"Okay, I will answer questions about  Health and wellness  What is your question?"}]},
        {"role":"user","parts":[{"text":user_query}]}
    ]
)

print("\n\n---------------------\n\n")

print(response.text)



---------------------


Based on the provided text, here are some food options for breakfast, lunch, and dinner, keeping in mind that the data includes sample meal plans and mentions healthy eating habits:


**Breakfast:**

*   Cereals (50g)
*   Milk (100ml)
*   Pulses (20g)  
*   Tea or Coffee (as per the sample plan)
*   Idli (4 Nos), Dosa (3 Nos), Upma (1-1/2 Cup), Bread (4 slices), Porridge (2 Cups), or Corn flakes with milk (2 Cups)  (Options for non-vegetarians include substituting a pulse portion with egg/meat/chicken/fish.)


**Lunch:**

*   Cereals (100g)
*   Rice (1 Cup)
*   Pulses (20g)
*   Dhal (1/2 Cup)
*   Vegetables (150g total;  Veg. curry and salad)
*   Milk or Curd (100ml or 1/2 cup)
*   Pulkas (2 Nos)


**Dinner:**

*   Cereals (100g)
*   Rice (1 Cup)
*   Pulses (20g)
*   Dhal (1/2 Cup)
*   Phulkas (2 Nos)


**Important Considerations from the text:**

*   **Portion sizes:** Pay attention to the portion sizes specified.
*   **Variety:**  The text emphasizes eating 